In [1]:
#利用现有的深度学习库实现数据迭代
import numpy as np
import torch
from d2l import torch as d2l
from torch.utils import data#可以方便地对数据进行处理和加载

In [2]:
"""⽣成y=Xw+b+噪声"""


def synthetic_data(w, b, num_examples):
    # 随机生成标准差为0，均值为1，形状为(num_examples,len((w)）的正态分布数据，因为w是x（特征，x1，x2。。。）的系数，即它的长度一定和特征的长度一样
    x = torch.normal(0, 1, (num_examples, len((w))))
    # torch.matmul(x,w) 计算两个张量的矩阵乘积
    y = torch.matmul(x, w)+b
    # 添加噪声，提高模型的泛化能力
    y += torch.normal(0, 0.01, y.shape)
    return x, y.reshape((-1, 1))


In [3]:
#1.生成数据集
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)


In [4]:
#2.读取数据集
def load_array(data_arrays,batch_size,is_shuffle=True):#构造一个pytorch数据迭代器
    # TensorDataset用于将数据转换为PyTorch中的TensorDataset格式
    # *data_arrays是可变参数，用于接收多个Tensor类型的数据，例如特征数据和标签数据。
    dataset = data.TensorDataset(*data_arrays)
    # DataLoader用于将数据集按照指定的batch size进行分批，同时还可以进行数据随机打乱、多进程加载等操作，方便高效地进行训练。
    return data.DataLoader(dataset,batch_size,shuffle=is_shuffle)#返回的是可迭代的对象，但不是迭代对象的地址
batch_size=10
data_iter=load_array((features,labels),batch_size,True)


# 在 Python 中，for 循环是基于迭代器（iterator）实现的
# 每次迭代 for 循环时，Python 会自动调用 iter() 函数获取数据的迭代器，并通过 next() 方法依次获取数据，直到迭代完成。
# 对于可迭代对象（如列表、元组、集合等），我们可以通过 iter() 函数将其转换为迭代器，然后通过 next() 方法依次获取元素。
# 在 PyTorch 中，DataLoader 返回的对象是一个可迭代对象，但不是迭代器，因此我们需要通过 iter() 函数将其转换为迭代器，才能通过 next() 方法获取数据。

next(iter(data_iter))


[tensor([[-0.2835,  0.8359],
         [ 0.1095,  1.0137],
         [ 0.9903,  1.2179],
         [-1.4117,  1.7299],
         [ 0.7212, -1.4881],
         [ 0.9078, -2.4126],
         [-0.3542,  0.2930],
         [-1.1048,  1.1259],
         [ 1.8264, -0.0479],
         [-0.0194,  0.2728]]),
 tensor([[ 0.8104],
         [ 0.9925],
         [ 2.0428],
         [-4.5008],
         [10.6954],
         [14.2273],
         [ 2.5048],
         [-1.8386],
         [ 8.0298],
         [ 3.2427]])]

In [5]:
#3、定义模型

#在PyTorch中，全连接层在Linear类中定义。 值得注意的是，我们将两个参数传递到nn.Linear中。 
#第一个指定输入特征形状，即2，第二个指定输出特征形状，输出特征形状为单个标量，因此为1
# Sequential类将多个层串联在一起。 当给定输入数据时，Sequential实例将数据传入到第一层， 然后将第一层的输出作为第二层的输入，以此类推。
from torch import nn
# 这个模型 net 是一个由单个线性层组成的顺序容器(nn.Sequential)。
# 这个线性层将输入张量的大小从 2 维度降到 1 维度，输入张量的形状应该是 (batch_size, 2)，输出张量的形状应该是 (batch_size, 1)。
net = nn.Sequential(nn.Linear(2, 1))
# net[0] 表示 net 这个顺序容器中的第一个子模块，因为在这个例子中，net 中只包含了一个子模块，即线性层(nn.Linear)。所以 net[0] 就是这个线性层。
#net[0].weight 是一个形状为(output_features_number, input_features_number) 的张量
#net[0].bias 是一个形状为(output_features_number,) 的张量


In [6]:
#4、初始化模型参数
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)


tensor([0.])

In [7]:
#5、定义损失函数
loss=nn.MSELoss(reduction='sum')#默认参数是mean，如果是sum 则模型的学习率要除以batch_size，这样lr就与batch_size解耦了

In [8]:
#6、定义优化算法
# net.parameters()获取模型的参数，用于SGD优化
trainer=torch.optim.SGD(net.parameters(),lr=0.03)

In [9]:
#7、训练
num_epoch=3
for epoch in range(num_epoch):
    for x,y in data_iter:
        #前向传播计算损失函数
        l=loss(net(x),y)
        #梯度清0
        trainer.zero_grad()
        #反向传播计算梯度
        l.backward()
        # trainer.step()表示优化器会根据当前的梯度值和学习率等超参数，对模型参数进行更新，使得模型的输出结果更加接近于真实标签值。
        # 注意的是，执行 trainer.step() 前，我们需要先执行 trainer.zero_grad() 将模型参数的梯度清零，
        # 否则模型参数的梯度会在每个批次的反向传播中被累积，导致参数更新出现错误。
        trainer.step()
    # 在每轮训练结束之后，计算模型在整个数据集上的损失。features 和 labels 是完整的数据集和标签。
    l=loss(net(features),labels)#梯度是否清0不影响求损失
    print(f'epoch{epoch+1},loss{l:f}')

epoch1,loss0.000563
epoch2,loss0.000089
epoch3,loss0.000089


In [10]:
w=net[0].weight.data
print('w的估计误差',true_w-w.reshape(true_w.shape))
b=net[0].bias.data
print('b的估计误差',true_b-b)

w的估计误差 tensor([ 2.8491e-05, -3.8195e-04])
b的估计误差 tensor([-6.4373e-05])
